In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

!cp /mydrive/Deepfake/train_sample_videos.zip  ./
!cp /mydrive/Deepfake/test_videos.zip  ./

!unzip ./train_sample_videos.zip -d ./train_videos/
!unzip ./test_videos.zip -d ./test_videos/

Archive:  ./train_sample_videos.zip
  inflating: ./train_videos/aagfhgtpmv.mp4  
  inflating: ./train_videos/aapnvogymq.mp4  
  inflating: ./train_videos/abarnvbtwb.mp4  
  inflating: ./train_videos/abofeumbvv.mp4  
  inflating: ./train_videos/abqwwspghj.mp4  
  inflating: ./train_videos/acifjvzvpm.mp4  
  inflating: ./train_videos/acqfdwsrhi.mp4  
  inflating: ./train_videos/acxnxvbsxk.mp4  
  inflating: ./train_videos/acxwigylke.mp4  
  inflating: ./train_videos/aczrgyricp.mp4  
  inflating: ./train_videos/adhsbajydo.mp4  
  inflating: ./train_videos/adohikbdaz.mp4  
  inflating: ./train_videos/adylbeequz.mp4  
  inflating: ./train_videos/aelfnikyqj.mp4  
  inflating: ./train_videos/aelzhcnwgf.mp4  
  inflating: ./train_videos/aettqgevhz.mp4  
  inflating: ./train_videos/aevrfsexku.mp4  
  inflating: ./train_videos/afoovlsmtx.mp4  
  inflating: ./train_videos/agdkmztvby.mp4  
  inflating: ./train_videos/agqphdxmwt.mp4  
  inflating: ./train_videos/agrmhtjdlk.mp4  
  inflating: ./trai

In [ ]:
TRAIN_DIR = './train_videos/'
TEST_DIR = './test_videos/'


BATCH_SIZE = 1
SCALE = 0.25
N_FRAMES = 200 

In [ ]:
!pip install facenet_pytorch

In [ ]:
import os
import glob
import json
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)




device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

cuda:0
Running on device: cuda:0


In [ ]:
class DetectionPipeline:
    """Pipeline class for detecting faces in the frames of a video file."""
    
    def __init__(self, detector, n_frames=None, batch_size=60, resize=None):
        """Constructor for DetectionPipeline class.
        
        Keyword Arguments:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            batch_size {int} -- Batch size to use with MTCNN face detector. (default: {32})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
    
    def __call__(self, filename):
        """Load frames from an MP4 video and detect faces.

        Arguments:
            filename {str} -- Path to video.
        """
        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Pick 'n_frames' evenly spaced frames to sample 
        # To implement if you want to skip some frames
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        faces = []
        
        #catch 200 frames don't care how many faces
        for j in range(self.n_frames):
          
            success = v_cap.grab()

      
              # Load frame
            success, frame = v_cap.retrieve()
            if not success:
                continue
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            
            # Resize frame to desired size
            if self.resize is not None:
                frame = frame.resize([int(d * self.resize) for d in frame.size])

            out = self.detector(frame)
            if out is not None:
              faces.extend(self.detector(frame))
          
            
            

        v_cap.release()
        return faces

In [ ]:
def process_faces(faces, feature_extractor, embedding_output = False):
    # Filter out frames without faces
    faces = [f for f in faces if f is not None]
    if len(faces) < 200:
        return None
    
    faces = torch.stack(faces).to(device)

    # Generate facial feature vectors using a pretrained model
    embeddings = feature_extractor(faces)

    # Calculate centroid for video and distance of each face's feature vector from centroid
    centroid = embeddings.mean(dim=0)
    x = (embeddings - centroid).norm(dim=1).cpu().numpy()
    
    if embedding_output == False:
      return x
    
    x = (embeddings-centroid).cpu().numpy()
  
    return x



In [ ]:
# Load face detector
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()

# Load facial recognition model
feature_extractor = InceptionResnetV1(pretrained='vggface2', device=device).eval()

# Define face detection pipeline
detection_pipeline = DetectionPipeline(detector=face_detector, n_frames=N_FRAMES, batch_size=BATCH_SIZE, resize=SCALE)

In [ ]:
# Get the paths of all train videos
all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))
all_test_videos = glob.glob(os.path.join(TEST_DIR, '*.mp4'))


In [ ]:
!cp /mydrive/Deepfake/archive.zip  ./

In [ ]:
!unzip ./archive.zip -d ./archive/

Archive:  ./archive.zip
  inflating: ./archive/metadata      


In [ ]:
metadata = pd.read_csv('./archive/metadata')
metadata = metadata[['filename','label']]
metadata.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,filename,label
0,hrundztjfb.mp4,FAKE
1,uyfnfvsxez.mp4,REAL
2,yknlciphae.mp4,FAKE
3,zfyczzssiz.mp4,FAKE
4,kggbciadwj.mp4,REAL


# Train Embedding Extraction

In [ ]:
train_frames_recorder = []
counter = 0
labels = []
embedding_data = None
realimagecounter = 0
fakeimagecounter = 0


with torch.no_grad():
    for video_path in tqdm(all_train_videos):

        filename = os.path.basename(video_path)

        # Detect all faces occur in the video
        faces = detection_pipeline(video_path)
        
        # Calculate the distances of all faces' feature vectors to the centroid
        result_embeddings = process_faces(faces, feature_extractor,embedding_output=True)
        
        
        if result_embeddings is None:
            continue

        if embedding_data is None:
          embedding_data = result_embeddings
        else:
          embedding_data = np.concatenate((embedding_data,result_embeddings),axis = 0)

       # needs to be optimized
       
        train_frames_recorder.append(len(result_embeddings))
        print("res : ",len(result_embeddings))

        label = 1 if metadata[metadata['filename'] == filename]['label'].values[0] == 'FAKE' else 0
        if label == 0:
          labels.extend(np.zeros((len(result_embeddings)),dtype = np.int8))
        else:
          labels.extend(np.ones((len(result_embeddings)),dtype = np.int8)) 

res :  202
res :  201
res :  201
res :  200
res :  200
res :  200
res :  204
res :  200
res :  206
res :  204
res :  200
res :  203
res :  405
res :  204
res :  200
res :  200
res :  200
res :  207
res :  200
res :  453
res :  200
res :  200
res :  210
res :  387
res :  257
res :  402
res :  223
res :  213
res :  202
res :  202
res :  201
res :  210
res :  202
res :  201
res :  403
res :  211
res :  207
res :  213
res :  200
res :  377
res :  200
res :  200
res :  200
res :  201
res :  384
res :  200
res :  200
res :  212
res :  201
res :  203
res :  200
res :  200
res :  200
res :  200
res :  200
res :  200
res :  200
res :  200
res :  200
res :  206
res :  400
res :  203
res :  200
res :  201
res :  200
res :  200
res :  437
res :  209
res :  200
res :  200
res :  200
res :  203
res :  200
res :  200
res :  200
res :  389
res :  403
res :  200
res :  205
res :  208
res :  200
res :  200
res :  200
res :  224
res :  203
res :  200
res :  200
res :  210
res :  202
res :  200
res :  200

Number of frames extraction, should be recorded on previous code line ...


In [ ]:
train_frames_recorder = []

with torch.no_grad():
    for video_path in tqdm(all_train_videos):
        filename = os.path.basename(video_path)
        # Detect all faces occur in the video
        faces = detection_pipeline(video_path)
        
        # Calculate the distances of all faces' feature vectors to the centroid
        result_embeddings = process_faces(faces, feature_extractor,embedding_output=True)

        if result_embeddings is None:
            continue       
        train_frames_recorder.append(len(result_embeddings))
        


In [ ]:
from numpy import savetxt
train_frames_recorder = np.array(train_frames_recorder)
savetxt('train_frames_recorder.csv', train_frames_recorder, delimiter=',')

In [ ]:
from numpy import savetxt
savetxt('trainvideos_np.csv', embedding_data, delimiter=',')

In [ ]:
!cp ./train_frames_recorder.csv /mydrive/Deepfake/train_frames_recorder.csv

In [ ]:
labels = np.array(labels)
savetxt('trainvideos_labels.csv',labels,delimiter=',')

In [ ]:
!cp ./trainvideos_np.csv /mydrive/Deepfake/trainvideos_np.csv
!cp ./trainvideos_labels.csv /mydrive/Deepfake/trainvideos_labels.csv

# Test Embedding Extraction

In [ ]:
df = pd.DataFrame(columns=['filename', 'distance', 'label'])

counter = 0
test_labels = []
test_embedding_data = None



with torch.no_grad():
    for video_path in tqdm(all_test_videos):

        filename = os.path.basename(video_path)

        # Detect all faces occur in the video
        faces = detection_pipeline(video_path)
        
        # Calculate the distances of all faces' feature vectors to the centroid
        result_embeddings = process_faces(faces, feature_extractor,embedding_output=True)
        
        
        if result_embeddings is None:
            continue

        if test_embedding_data is None:
          test_embedding_data = result_embeddings
        else:
          test_embedding_data = np.concatenate((test_embedding_data,result_embeddings),axis = 0)

       # needs to be optimized
       
        print("res : ",len(result_embeddings))

        label = 1 if metadata[metadata['filename'] == filename]['label'].values[0] == 'FAKE' else 0
        if label == 0:
          test_labels.extend(np.zeros((len(result_embeddings)),dtype = np.int8))
        else:
          test_labels.extend(np.ones((len(result_embeddings)),dtype = np.int8)) 

KeyboardInterrupt: ignored

In [ ]:
test_frames_recorder = []

with torch.no_grad():
    for video_path in tqdm(all_test_videos):
        filename = os.path.basename(video_path)
        # Detect all faces occur in the video
        faces = detection_pipeline(video_path)
        
        # Calculate the distances of all faces' feature vectors to the centroid
        result_embeddings = process_faces(faces, feature_extractor,embedding_output=True)

        if result_embeddings is None:
            continue       
        test_frames_recorder.append(len(result_embeddings))
        

In [ ]:
from numpy import savetxt
test_frames_recorder = np.array(test_frames_recorder)
savetxt('test_frames_recorder.csv', test_frames_recorder, delimiter=',')

In [ ]:
savetxt('testvideos_np.csv', test_embedding_data, delimiter=',')
test_labels = np.array(test_labels)
savetxt('testvideos_labels.csv',test_labels,delimiter=',')

In [ ]:
!cp ./testvideos_np.csv /mydrive/Deepfake/testvideos_np.csv
!cp ./testvideos_labels.csv /mydrive/Deepfake/testvideos_labels.csv

# Pull from Drive

In [ ]:
!cp /mydrive/Deepfake/trainvideos_labels.csv  ./trainvideos_labels.csv
!cp /mydrive/Deepfake/trainvideos_np.csv ./trainvideos_np.csv

!cp /mydrive/Deepfake/testvideos_labels.csv  ./testvideos_labels.csv
!cp /mydrive/Deepfake/testvideos_np.csv ./testvideos_np.csv

In [ ]:
!cp /mydrive/Deepfake/train_frames_recorder.csv ./train_frames_recorder.csv


In [ ]:
from numpy import loadtxt
# load array
embedding_data = np.array(loadtxt('trainvideos_np.csv', delimiter=','))
test_embedding_data = np.array(loadtxt('testvideos_np.csv', delimiter=','))

labels = np.array(loadtxt('trainvideos_labels.csv', delimiter=','))
test_labels = np.array(loadtxt('testvideos_labels.csv', delimiter=','))


In [ ]:
train_frames_recorder = np.array(loadtxt('train_frames_recorder.csv', delimiter=','))
train_frames_recorder.shape

(362,)

# Embedding Processing

Preprocessing

In [ ]:
embedding_data.shape,test_embedding_data.shape,labels.shape,test_labels.shape

((81704, 512), (72518, 512), (81704,), (72518,))

In [99]:
X = np.concatenate((embedding_data,test_embedding_data),axis = 0)
y = np.concatenate((labels,test_labels),axis = 0)
X.shape,y.shape

((154222, 512), (154222,))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((123377, 512), (30845, 512), (123377,), (30845,))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

X_train.mean(),X_val.mean()

(1.149292793035527e-18, 6.017501122312564e-05)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(.99)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_val = pca.transform(X_val)
X_train.shape,X_val.shape

((123377, 48), (30845, 48))

In [102]:
X_video_pred = scaler.transform(X)
X_video_pred = pca.transform(X_video_pred)
X_video_pred.shape

(154222, 48)

# SVM

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_predict = clf.predict(X_val)

In [59]:
y_predict_train = clf.predict(X_train)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, y_predict)

array([[ 7559,  3516],
       [ 1435, 18335]])

In [ ]:
(7559+18335)/(1435+3516+7559+18335)
# 0.8404 for real images accuracy
#0.8390 for fake images accuracy

0.8394877613875831

In [61]:
confusion_matrix(y_train, y_predict_train)

array([[32437, 11863],
       [ 4454, 74623]])

In [ ]:
(32437+74623) / (4454+11863+32437+74623)

#0.879226 real images accuracy
#  0.86 fake images accuracy

0.8677468247728507

In [ ]:
import pickle
filename = 'svm_embedding.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
!cp  ./svm_embedding.sav /mydrive/Deepfake/svm_embedding.sav 

# Accuracy On Videos

I should have recorded number of faces, in extraction phase so it will be repeated because of that inattention




In [ ]:
train_frames_recorder = train_frames_recorder.astype(np.int32)
test_frames_recorder = test_frames_recorder.astype(np.int32)
recorder = np.concatenate((train_frames_recorder,test_frames_recorder),axis = 0)
recorder.shape

(691,)

In [57]:
sum(recorder)

154222

In [104]:
acc = 0
true_counter = 0
false_counter = 0
accuracy_recorder = []
total_true_frames = 0

for record in recorder:

  predict = clf.predict(X_video_pred[acc:acc+record])
  values = y[acc:acc+record] #they all same anyways.
  mask = predict == values
  total_true = sum(mask)
  
  

  total_true_frames+=total_true

  if total_true/len(mask) >= 0.5:
    true_counter+=1
    accuracy_recorder.append(1)
  else:
    false_counter+=1
    accuracy_recorder.append(0)

  acc +=record


print("total true counter : ",true_counter)
print("total false counter : ",false_counter)

print("accuracy : ",true_counter / (true_counter+false_counter))
 

total true counter :  654
total false counter :  37
accuracy :  0.9464544138929089


In [107]:
print("total true frame when it is calculated video by video : ",total_true_frames)

print("total true frame when it is calculated frame by frame:",7559 + 18335 + 32437 + 74623)

total true frame when it is calculated video by video :  132954
total true frame when it is calculated frame by frame: 132954
